# Frends

    «Друзья» (англ. Friends) — американский ситком, повествующий о жизни шестерых друзей. Признан одним из лучших комедийных сериалов в истории американского телевидения и стал одним из наиболее знаменитых проектов 1990-х годов[2]. Сама постановка и творческая группа получили множество наград, в том числе шесть премий «Эмми» и премию «Золотой глобус»[3].

Че́ндлер Бинг (Мэттью Перри) — полное имя Че́ндлер Мюриэл Бинг

Чендлер родился в семье писательницы эротических романов Норы Тайлер Бинг и гомосексуала-трансвестита Чарльза Бинга. У Чендлера множество комплексов, от которых он отгораживается постоянными шутками. Работа Чендлера — анализ и реконфигурация данных, и в этой сфере он делает неплохую карьеру. Однако в конце сериала он бросает работу и начинает всё с нуля в рекламном бизнесе. Чендлеру не очень везёт в любви. По существу, у него было всего два более-менее серьёзных романа: с Дженис и с Моникой, на которой он женится в седьмом сезоне сериала.


 Диалоги формаировались на основании https://www.kaggle.com/datasets/amandam1/friends-scripts
 Диалог - это вопрос(контекст - предыдущая реплика)  - ответ персонажа

 Поиск ведется по наиболее релевантному контексту и отдается реплика персонажа


Нерелевантные примеры  формировались на соновании сценариев к фильмам взятым с   https://imsdb.com/all-scripts.html

# Подготовка

In [8]:
# %%capture

# %pip install -U peft
# %pip install -U trl
# %pip install -U bitsandbytes 

In [9]:
import os, torch, mlflow

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)

from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format, DataCollatorForCompletionOnlyLM
from dataclasses import dataclass
import random
import numpy as np
import os
import re
import json
import pandas as pd

In [10]:
# зафиксируем случайности
seed = 42

random.seed(seed)
np.random.seed(seed)

torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [11]:
@dataclass
class Config:
    model_name = "unsloth/Llama-3.2-1B-Instruct"  # со страницы запрещенной организации недоступна
    new_model = "llama-3.1-8b-chat-chandler"
    torch_dtype = torch.float16
    attn_implementation = "eager"
cfg = Config()

In [12]:
# настройки  

# Персонаж для диалога
persona = 'chandler'

output= f"{persona}.json"
other_output='other.json'

# Подготовка данных

берем данные и код обработки из первого ДЗ

In [13]:
# фунция обработки датасета Друзей

def extract_dialogs(filename, character_name=None):
    # Загрузка данных
    df = pd.read_csv(filename)
    
    # Группировка и очистка текста
    grouped = []
    prev_char, current_text = None, []
    
    for _, row in df.iterrows():
        # Очистка от \xa0 и лишних пробелов
        cleaned_line = row['line'].replace('\xa0', ' ').strip()
        if row['character'] == prev_char:
            current_text.append(cleaned_line)
        else:
            if prev_char is not None:
                grouped_text = ' '.join(current_text)
                grouped.append({'character': prev_char, 'text': grouped_text})
            prev_char = row['character']
            current_text = [cleaned_line]
    # Добавляем последнюю группу
    if prev_char is not None:
        grouped_text = ' '.join(current_text)
        grouped.append({'character': prev_char, 'text': grouped_text})
    
    # Формирование пар
    dialogs = []
    for i in range(len(grouped) - 1):
        question = grouped[i]['text']
        answer = grouped[i + 1]
        dialogs.append({
            'character': answer['character'],  # Ответивший
            'q': question,
            'a': answer['text']
        })
    
    # Фильтрация по отвечающему
    if character_name:
        dialogs = [d for d in dialogs if d['character'] == character_name]
    
    return dialogs

In [14]:
# получим все ответы персонажа с контекстом вопросом
person_dialogs = extract_dialogs('./data/friends.csv', persona)

# сохраним диалоги в файл
with open(output, 'w', encoding='utf-8') as file:
    json.dump(person_dialogs, file, ensure_ascii=False, indent=4)

# Первые 3 ответа персонажа
person_dialogs[:3]

[{'character': 'chandler',
  'q': "c'mon, you're going out with the guy! there's gotta be something wrong with him!",
  'a': 'all right joey, be nice.  so does he have a hump? a hump and a hairpiece?'},
 {'character': 'chandler',
  'q': "okay, everybody relax. this is not even a date. it's just two people going out to dinner and- not having sex.",
  'a': "sounds like a date to me. alright, so i'm back in high school, i'm standing in the middle of the cafeteria, and i realize i am totally naked."},
 {'character': 'chandler',
  'q': 'oh, yeah. had that dream.',
  'a': "then i look down, and i realize there's a phone... there."}]

In [15]:
# Функция обработки текстового формата сценариев и извлечения диалогов
def parse_dialogs(file_path, target_character=None):
    with open(file_path, 'r', encoding='utf-8') as f:
        script_text = f.read()    
    dialogs = []
    current_dialog = None
    previous_dialog = None
    actions_between = []
    
    character_pattern = re.compile(r'^([A-Z][A-Z\s\-\']+)(\s+\(.*?\))?\s*$')
    
    for line in script_text.split('\n'):
        line = line.strip()
        
        # Определяем строку с персонажем
        match = character_pattern.match(line)
        if match:
            # Сохраняем предыдущий диалог если он соответствует фильтру
            if current_dialog:
                if target_character is None or current_dialog['character'] == target_character:
                    dialogs.append({'character':current_dialog['character'],'q': ' '.join(current_dialog['context']), 'a' : ' '.join(current_dialog['lines'])})
                previous_dialog = current_dialog
                
            # Создаем новый диалог
            current_character = match.group(1).strip()
            current_dialog = {
                'character': current_character,
                'lines': [],
                'context': []
            }
            
            # Добавляем контекст: предыдущая реплика + действия
            if previous_dialog:
                prev_lines = f"{previous_dialog['character']}: {' '.join(previous_dialog['lines'])}"
                current_dialog['context'].append(prev_lines)
            current_dialog['context'].extend(actions_between)
            actions_between = []
            continue
        
        # Собираем действия между репликами
        if line.startswith('(') and line.endswith(')'):
            action = line[1:-1].strip()
            if current_dialog:
                current_dialog['context'].append(action)
            else:
                actions_between.append(action)
            continue
        
        # Собираем текст реплики
        if current_dialog and line:
            current_dialog['lines'].append(line)
    
    # Добавляем последний диалог если соответствует фильтру
    if current_dialog and (target_character is None or current_dialog['character'] == target_character):
        dialogs.append({'character':current_dialog['character'],'q': ' '.join(current_dialog['context']), 'a' : ' '.join(current_dialog['lines'])})
    
    return dialogs

In [16]:
# функция обработки файлов папки
def process_all_files(persona="THREEPIO", data_dir='data', output_file='all_answers.json'):
    all_dialogues = []
    
    for root, _, files in os.walk(data_dir):
        for file in files:
            if file.endswith('.txt'):
                input_file = os.path.join(root, file)                 
                print(f"Обработка файла: {input_file}")                
                dialogs = parse_dialogs(input_file, persona)
                all_dialogues.extend(dialogs)


    with open(output_file, 'w', encoding='utf-8') as file:
        json.dump(all_dialogues, file, ensure_ascii=False, indent=4)
        print(f"JSON-файл сохранен: {output_file}, записано строк: {len(all_dialogues)}")                
    print(f"Объединенный JSON-файл сохранен как: {output_file}")
    return all_dialogues

In [17]:
# Запуск обработки файлов сценариев из других вселенных для создания "неправильных" ответов
other_dialogs = process_all_files(persona = None, data_dir='data/other', output_file=other_output)

Обработка файла: data/other\KUNG FU PANDA.txt
Обработка файла: data/other\PEARL HARBOR.txt
Обработка файла: data/other\SW_Episode1.txt
Обработка файла: data/other\SW_Episode2.txt
Обработка файла: data/other\SW_Episode3.txt
Обработка файла: data/other\SW_Episode4.txt
Обработка файла: data/other\SW_Episode5.txt
Обработка файла: data/other\SW_Episode6.txt
Обработка файла: data/other\SW_Episode7.txt
Обработка файла: data/other\WILD WILD WEST.txt
JSON-файл сохранен: other.json, записано строк: 8474
Объединенный JSON-файл сохранен как: other.json


In [18]:
len(other_dialogs), other_dialogs[77]

(8474,
 {'character': 'SHIFU',
  'q': 'OOGWAY: One often meets his destiny on the road he takes to avoid it.',
  'a': "We have to do something. We can't just let him march on the valley, and take his revenge! He'll, he'll-- Oogway looks into the water of the moon pool."})

# Подготовка к обучению

In [52]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=cfg.torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    cfg.model_name,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=cfg.attn_implementation
)

In [53]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(cfg.model_name)
if hasattr(tokenizer, "chat_template") and tokenizer.chat_template is not None:
    tokenizer.chat_template = None  # Reset the chat template
model, tokenizer = setup_chat_format(model, tokenizer)
tokenizer.padding_side = 'right'
tokenizer.padding_token = '<|pad|>'

In [54]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model = get_peft_model(model, peft_config)

In [55]:
dataset_dict = load_dataset("json", data_files=output)
dataset = dataset_dict["train"] if "train" in dataset_dict else dataset_dict

In [23]:
#приведение к формату чата
def format_chat_template(row):
    row_json = [{"role": "user", "content": row["q"]},
               {"role": "assistant", "content": row["a"]}]
    text = tokenizer.apply_chat_template(row_json, tokenize=False)
    return text

In [56]:
# Создаем data collator, который определяет, откуда считать лосс
response_template = "assistant\n"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)


In [57]:
dataset_sh = dataset.shuffle(seed=2024).select(range(len(dataset)))
dataset_sh = dataset_sh.train_test_split(0.1)
dataset_sh

DatasetDict({
    train: Dataset({
        features: ['character', 'q', 'a'],
        num_rows: 7370
    })
    test: Dataset({
        features: ['character', 'q', 'a'],
        num_rows: 819
    })
})

In [ ]:
training_arguments = TrainingArguments(
    output_dir=cfg.new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,#ставим одну эпоху, со второй идет переобучение
    #max_steps=100,
    eval_strategy="steps",
    eval_steps=50,
    logging_steps=100,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=True,
    group_by_length=True,
    logging_dir='./logs',
    report_to=["mlflow"],  # Логирование только в MLflow        
    run_name="Llama-3.2-chandler",
)

# Обучение модели

In [59]:


trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_sh["train"],
    eval_dataset=dataset_sh["test"],
    peft_config=peft_config,
    processing_class = tokenizer,
    formatting_func=format_chat_template,
    args=training_arguments,
    data_collator=collator,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [60]:
trainer.train()

Step,Training Loss,Validation Loss
50,No log,3.166881
100,3.133500,3.100621
150,3.133500,3.093056
200,2.901700,3.071405
250,2.901700,3.089561
300,2.975000,2.990439
350,2.975000,2.981388
400,2.988000,3.022996
450,2.988000,2.978867
500,2.869100,2.982294


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=3685, training_loss=2.8355348253185424, metrics={'train_runtime': 2320.1983, 'train_samples_per_second': 3.176, 'train_steps_per_second': 1.588, 'total_flos': 1979661271670784.0, 'train_loss': 2.8355348253185424})

In [ ]:
path_to_save = "Llama-finetuned"
trainer.save_model(path_to_save)
model.save_pretrained(path_to_save)
tokenizer.save_pretrained(path_to_save)

c:\Users\rvv19\AppData\Local\Programs\Python\Python312\Lib\site-packages\peft\utils\save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


('Llama-finetuned\\tokenizer_config.json',
 'Llama-finetuned\\special_tokens_map.json',
 'Llama-finetuned\\tokenizer.json')

# Определение качества соответствия генерации   

Подготовим данные и обучим классификатор, который определяет является ли текст репликой Chandler, это позволит оценить качество обучения.

In [62]:
# Загрузка данных из JSON файла
df_json_ch = pd.read_json(output)
# Загрузка данных из CSV файла
df_json_other = pd.read_json(other_output)


# Объединение датафреймов
df = pd.concat([df_json_other, df_json_ch], ignore_index=True)

# Создание колонки 'chandler_answer': 1, если персонаж — "chandler", иначе 0
df['label'] = df['character'].apply(lambda x: 1 if str(x).lower() == 'chandler' else 0)

# Формирование итогового датафрейма с колонками "text" (из поля "a") и "chandler_answer"
result_df = df[['a', 'label']].rename(columns={'a': 'text'})

# Вывод первых строк итогового датафрейма
result_df.sample(10)

,text,label
47,"Dad Dad Dad, it was just a dream.",0
5804,"He will join us or die, my master. Vader kneel...",0
7530,Precinct 47. Here.,0
2099,Just feeling this good was worth it. In the ba...,0
16254,"uhm, we've got turkey grease.",1
7728,I could sure use those clothes now. She respon...,0
8251,"Yeah, it would. That's Loveless' Lair. 140 E...",0
3233,You have fought gallantly. Worthy of recogniti...,0
1730,"Your Highness, we are the Ambassadors, for the...",0
2518,"Yes, Master…",0


In [63]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
import evaluate
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import numpy as np
import torch

# Предполагается, что у вас уже есть датафрейм result_df с колонками 'text' и 'chandler_answer'
df = result_df.dropna(subset=['text', 'label'])

# Разделение данных
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Преобразование датафреймов в Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Загрузка токенизатора
tokenizer_classification = BertTokenizer.from_pretrained('bert-base-uncased')

# Функция токенизации
def tokenize_function(examples):
    return tokenizer_classification(examples["text"], padding="max_length", truncation=True, max_length=128)

# Токенизация датасетов
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Установка формата для PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Загрузка модели BERT для классификации (2 класса)
model_classific = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Параметры обучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=10,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    report_to=["mlflow"],  # Логирование только в MLflow     
    run_name="chandler_classifier",
)

# Загрузка метрики через evaluate
accuracy_metric = evaluate.load("accuracy")

# Функция для вычисления метрик
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

# Создание Trainer
trainer = Trainer(
    model=model_classific,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Обучение модели
trainer.train()

# Оценка модели
eval_results = trainer.evaluate()
print("Evaluation Results:", eval_results)



Map:   0%|          | 0/13330 [00:00<?, ? examples/s]

Map:   0%|          | 0/3333 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\rvv19\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.081300,0.246002,0.912991


Evaluation Results: {'eval_loss': 0.2460024058818817, 'eval_accuracy': 0.912991299129913, 'eval_runtime': 6.1188, 'eval_samples_per_second': 544.718, 'eval_steps_per_second': 68.151, 'epoch': 1.0}


# Сравнение работы обученной и не обученной модели

In [64]:
# выделение ответов
def generate_answer(model, prompt):
    chat = [
        { "role": "user:", "content": prompt },
    ]
    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=25, do_sample=True, temperature=0.5)
    conversation = tokenizer.decode(outputs[0])
    match = re.search(r"<\|im_start\|>assistant\s*(.*)", conversation, re.DOTALL)
    if match:
        assistant_answer = match.group(1).strip()
    else:
        assistant_answer = ("i has no answer!")
    return assistant_answer

In [65]:
# вопросы к chandler
q1 = "c'mon, you're going out with the guy! there's gotta be something wrong with him!"
q2 = "okay, everybody relax. this is not even a date. it's just two people going out to dinner and- not having sex."
q3 = "oh, yeah. had that dream."

## Тренированая модель

In [66]:
generate_answer(model, q1)

"no, i'm not going out with him! i'm just going out with the guy! he's not the guy!"

In [67]:
generate_answer(model, q2)

"i know, i know, i'm sorry. but i'm really, really, really, really, really, really,"

In [68]:
generate_answer(model, q3)

'oh, yeah? what happened? what happened? what happened? what happened? what happened? what happened? what happened?'

In [69]:
learned_anwer = []
for i in dataset_sh['test']['q']:
    learned_anwer.append(generate_answer(model, i))

In [70]:
from transformers import pipeline

# Загружаем пайплайн для классификации (предполагается, что модель fine-tuned сохранена в "./results")
classifier = pipeline("text-classification", model=model_classific, tokenizer=tokenizer_classification)



# Классифицируем каждую реплику
predictions = classifier(learned_anwer)

# Предполагаем, что модель обучалась так, что "LABEL_1" соответствует репликам Chandler,
# а "LABEL_0" — остальным.
num_chandler = sum(1 for pred in predictions if pred['label'] == "LABEL_1")
total = len(learned_anwer)
percentage = (num_chandler / total) * 100
percentage

Device set to use cuda:0


98.9010989010989

## Не тренированая модель

In [106]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=cfg.torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# Load model
casual_model = AutoModelForCausalLM.from_pretrained(
    cfg.model_name,
    quantization_config=bnb_config,
#     device_map="auto",
    attn_implementation=cfg.attn_implementation
)

tokenizer = tokenizer = AutoTokenizer.from_pretrained(cfg.model_name)
# tokenizer.padding_side = 'right'
# tokenizer.padding_token = '<|pad_token|>'

`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [108]:
if hasattr(tokenizer, "chat_template") and tokenizer.chat_template is not None:
    tokenizer.chat_template = None  # Reset the chat template
casual_model, tokenizer = setup_chat_format(casual_model, tokenizer)

In [109]:
generate_answer(casual_model, q1)

'"c\'mon, you\'re going out with the guy! there\'s gotta be something wrong with him!"<|eot_id|><|start_header_id|>assistant'

In [110]:
generate_answer(casual_model, q2)

'"okay, everybody relax. this is not even a date. it\'s just two people going out to dinner and- not'

In [85]:
generate_answer(casual_model, q3)

'oh, yeah. had that dream.<|eot_id|><|start_header_id|>assistant [PTM]<|eot_id|><|start_header_id|>assistant [PTM]<|eot_id|><|start_header_id|>assistant'

In [86]:
generate_answer(casual_model, q3)

'oh, yeah. had that dream.<|eot_id|><|start_header_id|>assistant [English]<|eot_id|><|start_header_id|>assistant [French]<|eot_id|><|start_header_id|>assistant [German'

In [111]:
unlearned_anwer = []
for i in dataset_sh['test']['q']:
    unlearned_anwer.append(generate_answer(casual_model, i))

In [112]:
from transformers import pipeline

# Загружаем пайплайн для классификации
classifier = pipeline("text-classification", model=model_classific, tokenizer=tokenizer_classification)



# Классифицируем каждую реплику
predictions = classifier(unlearned_anwer)

# Предполагаем, что модель обучалась так, что "LABEL_1" соответствует репликам Chandler,
# а "LABEL_0" — остальным.
num_chandler = sum(1 for pred in predictions if pred['label'] == "LABEL_1")
total = len(unlearned_anwer)
percentage = (num_chandler / total) * 100
percentage

Device set to use cuda:0


51.52625152625152

## выводы  

визуально - наблюдается повышение качества генерации модели после дообучения.  
классификатор - показывает рост качества ответов в 2 раза. а точнее 99% ответов модели можно считать репликами Чандлера.

была выбрана слишком "маленькая" модель на 1 миллиард параметров. до обучения она плохо ведет диалог что можно видеть на примерах   
после fine-tuning ситуация улучшилась, но все равно желательно взять модель не менее 7b




# Прото инференс на gradio

In [79]:
# загрузка сохраненнной модели

path_to_save = "Llama-finetuned"

import torch
from peft import AutoPeftModelForCausalLM
from trl import setup_chat_format

saved_model =  AutoPeftModelForCausalLM.from_pretrained(path_to_save, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(path_to_save)
if hasattr(tokenizer, "chat_template") and tokenizer.chat_template is not None:
    tokenizer.chat_template = None  # Reset the chat template
saved_model, tokenizer = setup_chat_format(saved_model, tokenizer)

In [80]:
# инференс на gradio
import gradio as gr


saved_model.to('cuda')

def chatbot(user_input, history):



    if history is None:
        history = []
    # Формируем строку диалога из истории переписки
    conversation = ""
    for user_text, bot_text in history:
        conversation += f"User: {user_text}\nBot: {bot_text}\n"
    conversation += f"User: {user_input}\nBot: "
    
    print(user_input)
    
    # Извлекаем ответ модели 
    bot_response = generate_answer(saved_model, user_input)
    # Обновляем историю переписки
    history.append((user_input, bot_response))
    return history

with gr.Blocks() as demo:
    chatbot_ui = gr.Chatbot(label="Чат с моделью")
    state = gr.State([])
    with gr.Row():
        txt = gr.Textbox(show_label=False, placeholder="Введите сообщение и нажмите Enter")
    txt.submit(chatbot, [txt, state], chatbot_ui)

demo.launch()


Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


# Подготовка к инференсу через LLAMA.CPP

Объединим слои LORA  в базовую модель

In [95]:
saved_model_mu = saved_model.merge_and_unload()

In [ ]:
saved_model_mu.save_pretrained('chandler')
tokenizer.save_pretrained('chandler')

In [103]:
tokenizer.save_pretrained('chandler')

('chandler\\tokenizer_config.json',
 'chandler\\special_tokens_map.json',
 'chandler\\tokenizer.json')

скачаем репозиторий llama.cpp для конвертации

In [99]:
!git clone https://github.com/ggerganov/llama.cpp.git

Cloning into 'llama.cpp'...


конвертируем в gguf для инеференса с помощью LLAMA.cpp

In [105]:
!python ./llama.cpp/convert_hf_to_gguf.py ./chandler  --outfile llama_chandler_1b_4q.gguf   --outtype q8_0 

INFO:hf-to-gguf:Loading model: chandler
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {32}
INFO:hf-to-gguf:gguf: loading model part 'model.safetensors'
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> Q8_0, shape = {2048, 128258}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.float16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.float16 --> Q8_0, shape = {8192, 2048}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.float16 --> Q8_0, shape = {2048, 8192}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.float16 --> Q8_0, shape = {2048, 8192}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.float16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.float16 --> Q8_0, shape = {2048, 512}
INFO:hf-to-gguf:blk.0.attn_output.weight,    torch.float16 --> Q8_0, shape = {2048, 2048}
INFO:hf